In [1]:
import os

In [15]:
from psst.case import read_matpower

import pandas as pd

from psst.model import build_model

mpc_files = []

root= './../../../matpower/'

for mpc in os.listdir(root):
    if mpc.startswith('case') and mpc.endswith('.m'):
        mpc_files.append(os.path.abspath(os.path.join(root, mpc)))

import getpass
        
USER = getpass.getuser()

In [16]:
mpc_files = [ '/Users/{}/Documents/GitRepos/matpower/case5.m'.format(USER),
 '/Users/{}/Documents/GitRepos/matpower/case30.m'.format(USER),
 '/Users/{}/Documents/GitRepos/matpower/case6ww.m'.format(USER),
 '/Users/{}/Documents/GitRepos/matpower/case9.m'.format(USER),
 '/Users/{}/Documents/GitRepos/matpower/case9target.m'.format(USER),
]


In [17]:

mpc_case = dict()
mpc_results = dict()

for mpc in mpc_files:

    try:
        case = read_matpower(mpc)

        model = build_model(case)

        model.solve(solver='xpress', verbose=False)

        s1 = (model.results.angles / 2 / pd.np.pi * 360).T[0]

        case_name = mpc.split('/')[-1].replace('.m', '')
        mpc_case[case_name] = [v for v in model.results.power_generated.values[0]]
        mpc_results[case_name] = dict()
        mpc_results[case_name]['line_power'] = model.results.line_power.values
        mpc_results[case_name]['angles'] = model.results.angles.values * 360 / 2 / pd.np.pi

        
    except Exception as e:
        print('Unable to solve {}\n{}'.format(mpc, e))

In [18]:
for k in mpc_case:
    print(k)
    mpc_case

case6ww
case9
case30
case5
case9target


In [19]:
from scipy.io import loadmat

In [20]:
matpower = loadmat('/Users/{}/Documents/GitRepos/matpower/results.mat'.format(USER))

In [21]:
def rmse(predictions, targets):
    predictions = pd.DataFrame(predictions).T[0]
    targets = pd.DataFrame(targets)[0]
    return np.sqrt(((predictions - targets) ** 2).mean())

In [22]:
import numpy as np

In [23]:
print('case30', rmse(mpc_results['case30']['angles'], matpower['a30']))
print('case9', rmse(mpc_results['case9']['angles'], matpower['a9']))
print('case6ww', rmse(mpc_results['case6ww']['angles'], matpower['a6ww']))
print('case5', rmse(mpc_results['case5']['angles'], matpower['a5']))
print('case9target', rmse(mpc_results['case9target']['angles'], matpower['a9target']))


('case30', 4.0528596704901583)
('case9', 6.8282476828978767e-15)
('case6ww', 1.8831566595121686e-15)
('case5', 1.7406087768693023e-15)
('case9target', 0.6548200235238163)


In [24]:
print('case30', rmse(mpc_results['case30']['line_power'], matpower['lp30']))
print('case9', rmse(mpc_results['case9']['line_power'], matpower['lp9']))
print('case6ww', rmse(mpc_results['case6ww']['line_power'], matpower['lp6ww']))
print('case5', rmse(mpc_results['case5']['line_power'], matpower['lp5']))
print('case9target', rmse(mpc_results['case9target']['line_power'], matpower['lp9target']))


('case30', 5.7540935602070547e-14)
('case9', 2.5646338774393149e-14)
('case6ww', 2.3228879851839006e-14)
('case5', 2.4123597344804154e-13)
('case9target', 5.4935306698989068)
